## Import modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from streetview.streetview import search, api, download
from streetview.streetview import tools
import geopandas as gpd
import json
import os
import math
import requests
import geometry
from PIL import Image

## Set input and output folders

In [3]:
# input_folder='data'
input_folder='output\Wagga'
out_folder='output\Wagga\Panos'

## Set Google API key
Create a config.json file containing below to save your API key:  
{  
    "api_key": your_API_Key  
}  

In [4]:
with open('config.json', 'r') as f:
    config = json.load(f)
GOOGLE_MAPS_API_KEY = config['api_key']

## Load GNAF points
We use the points data at Wagga Wagga for demonstration:

In [ ]:
# add_points=os.path.join(input_folder,"Final_Wagga.shp")
add_points=os.path.join(input_folder,"Final_Wagga_training_samples.geojson")
gdf_points=gpd.read_file(add_points).to_crs('epsg:4326')
gdf_points.head()

### View points

In [ ]:
gdf_points.explore(column='USAGE')

### Filter by usage - keeping only residential

In [7]:
# gdf_points=gdf_points[gdf_points['USAGE'] == 'Residential']
# gdf_points

Test workflow with first point:

In [8]:
i=0
lon=gdf_points.geometry.iloc[i].x
lat=gdf_points.geometry.iloc[i].y

## Get ID of closest panorama using Google API

In [ ]:
# Base URL for the Street View Metadata API (for metadata)
metadata_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={lat},{lon}&key={GOOGLE_MAPS_API_KEY}"

# Fetch the metadata
metadata_response = requests.get(metadata_url)
metadata = metadata_response.json()

# Check if metadata request is successful and contains panorama data
if metadata.get('status') == 'OK':  
    # Print metadata details
    print("Panorama Metadata:")
    for key, value in metadata.items():
        print(f"{key}: {value}")
else:
    print(f"Error fetching metadata. Status: {metadata.get('status')}")


## Search all available panorama images by coordinates using streetview
The photos on Google street view are panoramas. Each parnorama has its own unique ID. Retrieving photos is a two step process. First, you must translate GPS coordinates into panorama IDs. The following code retrieves a list of the closest panoramas:

In [ ]:
panos = search.search_panoramas(lat=lat, lon=lon)
panos

Sort panoramas by distance to the building point and keep only top three panoramas:

In [11]:
# panos_filtered={'dists':[],'panos':[]}
# for pano in panos:
#     dist=math.sqrt((lat-pano.lat)**2+(lon-pano.lon)**2)
#     panos_filtered['panos'].append(pano)
#     panos_filtered['dists'].append(dist)
# sorted_data = sorted(zip(panos_filtered['dists'], panos_filtered['panos']), key=lambda x: x[0])
# dists_sorted, panos_sorted = zip(*sorted_data[:3])
# panos_filtered['dists'] = list(dists_sorted)
# panos_filtered['panos'] = list(panos_sorted)
# panos_filtered

## Find pano matching the API result to get metadata including:
- latitude and longitude
- heading/bearing angle
- pitch and roll angles (how are they retrieved?)

In [ ]:
for pano in panos:
    if pano.pano_id==metadata.get('pano_id'):
        heading=pano.heading
        pano_id=metadata.get('pano_id')
        print('Found matching pano with id of ',pano_id)
        print('heading angle of ',heading)

## Record metadata from query results

In [ ]:
metadata_all={}
all_panos={}
new_id=metadata.get('pano_id')
for pano in panos:
    all_panos[pano.pano_id]={'location': {'lat': pano.lat, 'lng': pano.lon},
                             'heading':pano.heading,
                             'pitch':pano.pitch,
                             'roll':pano.roll,
                             'date':pano.date,
                             'elevation':pano.elevation}
metadata_all[new_id]={'date':metadata.get('date'),
                      'location':metadata.get('location'),
                      'house_location':{'lat': lat, 'lng': lon},
                      'status':metadata.get('status'),
                      'All_panos':all_panos}
metadata_all

## Download pano image
The code below uses the panorama ids to download streetview images and the panorama in an asynchronous context. Street View Static API images can be returned in any size up to 640 x 640 pixels.

In [ ]:
zoom_level=3 # pano dimension (excluding user-uploaded data): 2^zoom*512, 2^(zoom-1)*512
output_pano=os.path.join(out_folder,pano_id+".jpg")
if not os.path.exists(output_pano):
    print('Downloading panorama image...')
    # meta = api.get_panorama_meta(pano_id=pano_id, api_key=GOOGLE_MAPS_API_KEY)
    gsv_pano = await download.get_panorama_async(pano_id=pano_id,
                                                zoom=zoom_level)
    # Crop the black border at the bottom and right of the panorama
    gsv_pano = tools.crop_bottom_and_right_black_border(gsv_pano)
    gsv_pano.save(output_pano, "jpeg")
else: # avoid redownloading existing pano
    print('Panorama image exists, reading it...')
    gsv_pano=Image.open(output_pano)
gsv_pano

## Download street view images (optional)

In [ ]:
# heading_street_view=location['camera_house_bearing']
# if heading_street_view<0:
#     heading_street_view=heading_street_view+360
    
# gsv_image = api.get_streetview(pano_id=pano_id,
#                                heading=heading_street_view,
#                                pitch=0,
#                                api_key=GOOGLE_MAPS_API_KEY)
# gsv_image.save(os.path.join(out_folder,"street_view_"+pano_id+".jpg"), "jpeg")

## Save metadata

In [15]:
out_file=os.path.join(out_folder,'metadata_all.json')
with open(out_file, 'w') as json_file:
    json.dump(metadata_all, json_file, indent=4)

## Batch downloading

In [ ]:
metadata_all={}
for i in range(len(gdf_points)):
# for i in range(10):
    lon=gdf_points.geometry.iloc[i].x
    lat=gdf_points.geometry.iloc[i].y

    # Base URL for the Street View Metadata API (for metadata)
    metadata_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={lat},{lon}&key={GOOGLE_MAPS_API_KEY}"

    try:
        # Fetch id of closest pano
        metadata_response = requests.get(metadata_url)
        metadata = metadata_response.json()

        # Check if metadata request is successful and contains panorama data
        if metadata.get('status') == 'OK':  
            # Print metadata details
            print("Panorama Metadata:")
            for key, value in metadata.items():
                print(f"{key}: {value}")
        else:
            print(f"Error fetching metadata. Status: {metadata.get('status')}")

        # get metadata of all panos
        panos = search.search_panoramas(lat=lat, lon=lon)
        all_panos_meta={}
        for pano in panos:
            all_panos_meta[pano.pano_id]={'location': {'lat': pano.lat, 'lng': pano.lon},
                             'heading':pano.heading,
                             'pitch':pano.pitch,
                             'roll':pano.roll,
                             'date':pano.date,
                             'elevation':pano.elevation}
            if pano.pano_id==metadata.get('pano_id'):
                heading=pano.heading
                pano_id=metadata.get('pano_id')
                print('Found matching pano with id of ',pano_id)
                print('heading angle of ',heading)
        metadata_all[pano_id]={'date':metadata.get('date'),
                      'location':metadata.get('location'),
                      'house_location':{'lat': lat, 'lng': lon},
                      'status':metadata.get('status'),
                      'All_panos':all_panos_meta}
        
        # download closest pano
        output_pano=os.path.join(out_folder,pano_id+".jpg")
        if not os.path.exists(output_pano):
            print('Downloading panorama image...')
            # meta = api.get_panorama_meta(pano_id=pano_id, api_key=GOOGLE_MAPS_API_KEY)
            gsv_pano = await download.get_panorama_async(pano_id=pano_id,
                                                        zoom=zoom_level)
            # Crop the black border at the bottom and right of the panorama
            gsv_pano = tools.crop_bottom_and_right_black_border(gsv_pano)
            gsv_pano.save(output_pano, "jpeg")
        else: # avoid redownloading existing pano
            print('Panorama image exists, skipping it...')

    except Exception as e:  # Catch any other unexpected errors:
        print(f"An error occurred: {e}, skipping...")

## Save final metadata

In [24]:
# save metadata
with open(out_file, 'w') as json_file:
    json.dump(metadata_all, json_file, indent=4)

In [ ]:
len(metadata_all)